<table style="width:100%; border-collapse: collapse;">
  <tr>
    <td style="width:20%; vertical-align:middle;">
      <img src="LogoUVG.png" width="400"/>
    </td>
    <td style="text-align:left; vertical-align:middle;">
      <h2 style="margin-bottom: 0;">Universidad del Valle de Guatemala - UVG</h2>
      <h3 style="margin-top: 0;">Facultad de Ingeniería - Computación</h3>
      <p style="font-size: 16px; margin-bottom: 0; margin-top: -20px">
        <strong>Curso:</strong> CC3104 - Aprendizaje por Refuerzo 
        <strong>Sección:</strong> 10
      </p>
      <p style="font-size: 16px; margin: 0;"><strong>Laboratorio 7:</strong> Policy Gradients Methods</p>
      <br>
      <p style="font-size: 15px; margin: 0;"><strong>Autores:</strong></p>
      <ul style="margin-top: 5px; padding-left: 20px; font-size: 15px;">
        <li>Diego Alexander Hernández Silvestre - <strong>21270</strong></li>
        <li>Linda Inés Jiménez Vides - <strong>21169</strong></li>
        <li>Mario Antonio Guerra Morales - <strong>21008</strong></li>
      </ul>
    </td>
  </tr>
</table>

## 📝 Task 1

**Explique la diferencia entre los métodos de aprendizaje de refuerzo basados en valores y en políticas. ¿Por qué los métodos de gradiente de políticas son especialmente útiles para entornos con espacios de acción continua?**

En lo que se refiere a los métodos value-based, ellos aprenden una función de valor y conforme van eligiendo acciones que maximizan dicho valor, también van derivando su política. Mientras que los policy-based aprenden directamente de los parámetros de una política, optimizando así el retorno esperado con la gradiente de la política. Es debido a esto que los métodos de gradiente en políticas son útiles en espacios de acción continua, porque la acción se muestrea de una distribución diferenciable y el entrenamiento ajusta sus parámetros por la gradiente, evitando maximizar explícitamente sobre un espacio continuo.

## 📝 Task 2

**Implemente una versión simple del algoritmo REINFORCE. Use un entorno simple, como CartPole-v1 de OpenAI Gym, para entrenar a un agente. La función de valor estimado debe utilizar un aproximador de función lineal.**

Pasos a considerar:

* Inicialice la política (por ejemplo, una red neuronal o una política softmax).

* Simule episodios y obtenga recompensas.

* Calcule el rendimiento descontado para cada par de estado-acción.

* Actualice los parámetros de la política utilizando actualizaciones de gradiente de política.

* Agregue una línea base (función de valor estimado) para reducir la varianza.

# Librerías + config

In [15]:
import gymnasium as gym
import torch, torch.nn as nn, torch.optim as optim
import numpy as np
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Inicialización de la política

In [ ]:
class PolicyNet(nn.Module):
    def __init__(self, obsDim, actDim, hidden=128):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(obsDim, hidden), nn.ReLU(),
            nn.Linear(hidden, actDim)
        )
    def forward(self, x):
        return self.net(x)

def initPolicy(envId="CartPole-v1", policyLr=3e-3):
    env = gym.make(envId)
    obsDim = env.observation_space.shape[0] # type: ignore
    actDim = env.action_space.n # type: ignore
    policy = PolicyNet(obsDim, actDim).to(DEVICE)
    optPi  = optim.Adam(policy.parameters(), lr=policyLr)
    return env, policy, optPi, obsDim, actDim


### Método para simulación de episodios y obtención de recompensas

In [17]:
@torch.no_grad()
def simulateEpisode(env, policy, maxSteps=500):
    obs, _ = env.reset()
    states, actions, rewards = [], [], []
    for _ in range(maxSteps):
        s = torch.tensor(obs, dtype=torch.float32, device=DEVICE).unsqueeze(0)
        logits = policy(s)
        dist = torch.distributions.Categorical(logits=logits)
        a = dist.sample()
        obs, r, terminated, truncated, _ = env.step(a.item())
        done = terminated or truncated  
        states.append(s.squeeze(0).cpu().numpy())
        actions.append(a.item())
        rewards.append(float(r))
        if done: break

    return np.array(states, np.float32), np.array(actions), np.array(rewards, np.float32)


### Cálculo del rendimiento descontado para cada estado-acción

In [ ]:
def computeDiscountedReturns(rewards, gamma=0.99):
    T = len(rewards)
    G = np.zeros(T, dtype=np.float32)
    running = 0.0
    for t in reversed(range(T)):
        running = rewards[t] + gamma * running
        G[t] = running
    return G


### Actualización de los parámetros de la política utilizando actualizaciones de gradiente de política

In [ ]:
def updatePolicy(policy, optPi, statesNp, actionsNp, advantagesT):
    statesT  = torch.tensor(statesNp,  dtype=torch.float32, device=DEVICE)
    actionsT = torch.tensor(actionsNp, dtype=torch.int64,   device=DEVICE)

    dist = torch.distributions.Categorical(logits=policy(statesT))
    logProbs = dist.log_prob(actionsT)
    lossPi = -(logProbs * advantagesT.detach()).mean()

    optPi.zero_grad()
    lossPi.backward()
    optPi.step()
    return float(lossPi.item())


### Agregación de línea base/función de valor estimado para reducción de la varianza

In [ ]:
class ValueLinear(nn.Module):
    """Baseline lineal: V(s) = w^T s + b."""
    def __init__(self, obsDim):
        super().__init__()
        self.v = nn.Linear(obsDim, 1)
    def forward(self, x):
        return self.v(x).squeeze(-1)

def initBaseline(obsDim, valueLr=5e-3):
    valueFn = ValueLinear(obsDim).to(DEVICE)
    optV = optim.Adam(valueFn.parameters(), lr=valueLr)
    return valueFn, optV

def computeAdvantages(valueFn, statesNp, returnsNp):
    with torch.no_grad():
        vals = valueFn(torch.tensor(statesNp, dtype=torch.float32, device=DEVICE))
    adv = torch.tensor(returnsNp, device=DEVICE) - vals
    adv = (adv - adv.mean()) / (adv.std() + 1e-8)
    return adv

def updateValue(valueFn, optV, statesNp, returnsNp):
    statesT  = torch.tensor(statesNp,  dtype=torch.float32, device=DEVICE)
    returnsT = torch.tensor(returnsNp, dtype=torch.float32, device=DEVICE)
    pred = valueFn(statesT)
    lossV = ((pred - returnsT) ** 2).mean()

    optV.zero_grad()
    lossV.backward()
    optV.step()
    return float(lossV.item())


### Entrenamiento

In [21]:
def trainReinforce(envId="CartPole-v1", gamma=0.99, maxEpisodes=750, maxSteps=500, logEvery=10):
    env, policy, optPi, obsDim, actDim = initPolicy(envId)
    valueFn, optV = initBaseline(obsDim)
    history = []

    for ep in range(1, maxEpisodes+1):
        states, actions, rewards = simulateEpisode(env, policy, maxSteps)
        returnsNp = computeDiscountedReturns(rewards, gamma)
        advantagesT = computeAdvantages(valueFn, states, returnsNp)

        lossPi = updatePolicy(policy, optPi, states, actions, advantagesT)
        lossV  = updateValue(valueFn, optV, states, returnsNp)

        epReturn = float(rewards.sum()); history.append(epReturn)
        if ep % logEvery == 0:
            avg20 = np.mean(history[-20:]) if len(history) >= 20 else np.mean(history)
            print(f"EPISODE {ep:4d} - RETURN: {epReturn:6.1f} - AVG: {avg20:6.1f} - Lπ: {lossPi:.4f} - Lv: {lossV:.4f}")
    env.close()

In [22]:
trainReinforce()

EPISODE   10 - RETURN:   13.0 - AVG:   27.6 - Lπ: 0.0306 - Lv: 53.0284
EPISODE   20 - RETURN:   37.0 - AVG:   28.3 - Lπ: 0.0042 - Lv: 347.7358
EPISODE   30 - RETURN:   21.0 - AVG:   36.3 - Lπ: 0.0272 - Lv: 125.1768
EPISODE   40 - RETURN:   22.0 - AVG:   43.2 - Lπ: 0.0383 - Lv: 134.8622
EPISODE   50 - RETURN:   92.0 - AVG:   43.3 - Lπ: -0.0354 - Lv: 1474.0721
EPISODE   60 - RETURN:   38.0 - AVG:   47.5 - Lπ: 0.0153 - Lv: 356.6764
EPISODE   70 - RETURN:  116.0 - AVG:   55.5 - Lπ: 0.0296 - Lv: 2016.9814
EPISODE   80 - RETURN:   39.0 - AVG:   60.6 - Lπ: 0.0241 - Lv: 368.4306
EPISODE   90 - RETURN:   99.0 - AVG:   74.8 - Lπ: 0.0210 - Lv: 1601.1178
EPISODE  100 - RETURN:  107.0 - AVG:  105.2 - Lπ: 0.0024 - Lv: 1780.9792
EPISODE  110 - RETURN:  177.0 - AVG:  115.3 - Lπ: 0.0002 - Lv: 3303.7903
EPISODE  120 - RETURN:  169.0 - AVG:  134.4 - Lπ: 0.0092 - Lv: 3147.3569
EPISODE  130 - RETURN:  260.0 - AVG:  186.9 - Lπ: -0.0013 - Lv: 4630.5469
EPISODE  140 - RETURN:  219.0 - AVG:  223.2 - Lπ: 0.0129

Los resultados reflejan el proceso de aprendizaje que realiza el agente mediante REINFORCE con baseline en el entorno de CartPole. Al inicio, los retornos fueron bajos e inestables, lo que muestra cómo se comporta la fase de exploración (inicial) en la que la política aún era casi aleatoria. Con el paso de los episodios, los promedios comenzaron a incrementarse hasta alcanzar valores cercanos al máximo posible (500), evidenciando que la política logró aprender a mantener el equilibrio del palo de manera consistente. Aunque se observan caídas puntuales, el desempeño general confirma que el agente alcanzó y mantuvo el criterio de “resolver” el entorno, estabilizando su comportamiento a través de la retroalimentación basada en retornos y la corrección de varianza mediante la función de valor.